Establish the connection to the database or the content from the url

In [ ]:
# import yaml
# from sqlalchemy import create_engine
# from langchain_community.utilities import SQLDatabase

# with open('../config/config.yml', 'r') as file:
#     config = yaml.safe_load(file)

# mysql_config = config.get('mysql')

# username = mysql_config.get('username')
# password = mysql_config.get('password')
# host = mysql_config.get('host')
# port = mysql_config.get('port')
# database = mysql_config.get('database')

# connection_string = f"mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}"
# engine = create_engine(connection_string)

# db = SQLDatabase(engine)

In [2]:
from pyprojroot import here

from sqlalchemy import create_engine
from langchain_community.utilities import SQLDatabase

database_path = here("data/sqldb.db")

connection_string = f"sqlite:///{database_path}"
engine = create_engine(connection_string, echo=True)
db = SQLDatabase(engine)

2025-02-27 10:29:07,929 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-27 10:29:07,929 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-02-27 10:29:07,930 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-27 10:29:07,931 INFO sqlalchemy.engine.Engine ROLLBACK
2025-02-27 10:29:07,931 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-27 10:29:07,931 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-02-27 10:29:07,931 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-27 10:29:07,932 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_temp_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-02-27 10:29:07,932 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-27 10:29:07,932 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("Album")
2025-02-27 10:29:07,932 I

Create the connection to the database by inputing the api_key and making the connection to the clod provider

In [1]:
from getpass import getpass
import os
from langchain.chat_models import init_chat_model

os.environ["OPENAI_API_KEY"] = getpass()
llm = init_chat_model("gpt-4o-mini", model_provider="openai")

Create the fileds for the State that is going to be parsed among the agent pipeline

In [3]:
from typing_extensions import TypedDict


class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str

# sets the fields that are passed among the states

Populate the model with the predifined prompts designed to tell the model in what direction it should navigate the response

In [4]:
from langchain import hub

query_prompt_template = hub.pull("langchain-ai/sql-query-system-prompt")

assert len(query_prompt_template.messages) == 1
query_prompt_template.messages[0].pretty_print()

/Users/mark/Documents/Research/LLM_MODEL/.venv/lib/python3.11/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


================================ System Message ================================

Given an input question, create a syntactically correct {dialect} query to run to help find the answer. Unless the user specifies in his question a specific number of examples they wish to obtain, always limit your query to at most {top_k} results. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

Only use the following tables:
{table_info}

Question: {input}


Populate the generated string ouptuted from the previous cell and fill it with an appropriate query

In [5]:
from typing_extensions import Annotated


class QueryOutput(TypedDict):
    """Generated SQL query."""

    query: Annotated[str, ..., "Syntactically valid SQL query."]


def write_query(state: State):
    """Generate SQL query to fetch information."""
    prompt = query_prompt_template.invoke(
        {
            "dialect": db.dialect,
            "top_k": 10,
            "table_info": db.get_table_info(),
            "input": state["question"],
        }
    )
    structured_llm = llm.with_structured_output(QueryOutput)
    result = structured_llm.invoke(prompt)
    return {"query": result["query"]}

In [6]:
write_query({"question": "How many Employees are there?"})

2025-02-27 10:29:29,525 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-27 10:29:29,526 INFO sqlalchemy.engine.Engine SELECT "Artist"."ArtistId", "Artist"."Name" 
FROM "Artist"
 LIMIT ? OFFSET ?
2025-02-27 10:29:29,527 INFO sqlalchemy.engine.Engine [generated in 0.00164s] (3, 0)
2025-02-27 10:29:29,530 INFO sqlalchemy.engine.Engine ROLLBACK
2025-02-27 10:29:29,532 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-27 10:29:29,533 INFO sqlalchemy.engine.Engine SELECT "Employee"."EmployeeId", "Employee"."LastName", "Employee"."FirstName", "Employee"."Title", "Employee"."ReportsTo", "Employee"."BirthDate", "Employee"."HireDate", "Employee"."Address", "Employee"."City", "Employee"."State", "Employee"."Country", "Employee"."PostalCode", "Employee"."Phone", "Employee"."Fax", "Employee"."Email" 
FROM "Employee"
 LIMIT ? OFFSET ?
2025-02-27 10:29:29,533 INFO sqlalchemy.engine.Engine [generated in 0.00145s] (3, 0)
2025-02-27 10:29:29,536 INFO sqlalchemy.engine.Engine ROLLBACK
2025-02

{'query': 'SELECT COUNT(*) AS EmployeeCount FROM Employee;'}

In [ ]:
# TODO: find the way to create a technique to choose only the most appropriate tables from the list of tables the user have at the moment

Creating the method (tool) that is responsible for the running the generated query against the database

In [7]:
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool

# notice that the instance of the state is passed along all the methods within a chain
def execute_query(state: State):
    """Execute SQL query."""
    execute_query_tool = QuerySQLDatabaseTool(db=db)
    return {"result": execute_query_tool.invoke(state["query"])}

The way how it can be called: 

In [8]:
execute_query({"query": "SELECT COUNT(*) AS employee_count FROM employee;"})

2025-02-27 10:29:54,730 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-27 10:29:54,731 INFO sqlalchemy.engine.Engine SELECT COUNT(*) AS employee_count FROM employee;
2025-02-27 10:29:54,731 INFO sqlalchemy.engine.Engine [generated in 0.00044s] ()
2025-02-27 10:29:54,733 INFO sqlalchemy.engine.Engine COMMIT


{'result': '[(8,)]'}

The next tool is responsible for generation of the human like response. In future generations, should be changed to generation of the grpah

In [9]:
def generate_answer(state: State):
    """Answer question using retrieved information as context."""
    prompt = (
        "Given the following user question, corresponding SQL query, "
        "and SQL result, answer the user question.\n\n"
        f'Question: {state["question"]}\n'
        f'SQL Query: {state["query"]}\n'
        f'SQL Result: {state["result"]}'
    )
    response = llm.invoke(prompt)
    return {"answer": response.content}

Creating the pipeline of the chains to create the end-to-end agent

In [10]:
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence(
    [write_query, execute_query, generate_answer]
)
graph_builder.add_edge(START, "write_query") #setting write_query as the starting point
graph = graph_builder.compile()

Example of the possible interaction by the human in the chain

In [11]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
# pay attention to the thing that the human interaction is inserted before running the execute_query
graph = graph_builder.compile(checkpointer=memory, interrupt_before=["execute_query"])

config = {"configurable": {"thread_id": "1"}}

Below is the possible way of how it can be integrated to the loop via wrapping the code in teh rty-catch block

In [12]:
for step in graph.stream(
    {"question": "How many employees are there?"},
    config,
    stream_mode="updates",
):
    print(step)

# loop should be prevented before running the query and the user's answer is passed forward

try:
    user_approval = input("Do you want to go to execute query? (yes/no): ")
except Exception:
    user_approval = "no"

if user_approval.lower() == "yes":
    # If approved, continue the graph execution
    for step in graph.stream(None, config, stream_mode="updates"):
        print(step)
else:
    print("Operation cancelled by user.")

2025-02-27 10:30:08,664 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-27 10:30:08,665 INFO sqlalchemy.engine.Engine SELECT "Artist"."ArtistId", "Artist"."Name" 
FROM "Artist"
 LIMIT ? OFFSET ?
2025-02-27 10:30:08,666 INFO sqlalchemy.engine.Engine [cached since 39.14s ago] (3, 0)
2025-02-27 10:30:08,666 INFO sqlalchemy.engine.Engine ROLLBACK
2025-02-27 10:30:08,667 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-27 10:30:08,668 INFO sqlalchemy.engine.Engine SELECT "Employee"."EmployeeId", "Employee"."LastName", "Employee"."FirstName", "Employee"."Title", "Employee"."ReportsTo", "Employee"."BirthDate", "Employee"."HireDate", "Employee"."Address", "Employee"."City", "Employee"."State", "Employee"."Country", "Employee"."PostalCode", "Employee"."Phone", "Employee"."Fax", "Employee"."Email" 
FROM "Employee"
 LIMIT ? OFFSET ?
2025-02-27 10:30:08,668 INFO sqlalchemy.engine.Engine [cached since 39.14s ago] (3, 0)
2025-02-27 10:30:08,669 INFO sqlalchemy.engine.Engine ROLLBACK
202